In [ ]:
#pip install datasets

In [6]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [7]:
from datasets import load_dataset
dataset = load_dataset("billsum", split = "test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


In [8]:
import pandas as pd
dataset.set_format("pandas")

df = dataset[0:]
df

,text,summary,title
0,SECTION 1. ENVIRONMENTAL INFRASTRUCTURE.\n\n ...,Amends the Water Resources Development Act of ...,To make technical corrections to the Water Res...
1,That this Act may be cited as the ``Federal Fo...,Federal Forage Fee Act of 1993 - Subjects graz...,Federal Forage Fee Act of 1993
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,. Merchant Marine of World War II Congression...,Merchant Marine of World War II Congressional ...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Modernization Act of 2004 - Ame...,To amend the Internal Revenue Code of 1986 to ...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Fair Access to Investment Research Act of 2016...,Fair Access to Investment Research Act of 2016
...,...,...,...
3264,SECTION 1. PLACEMENT PROGRAMS FOR FEDERAL EMPL...,Public Servant Priority Placement Act of 1995 ...,Public Servant Priority Placement Act of 1995
3265,SECTION 1. SHORT TITLE.\n\n This Act may be...,Sportsmanship in Hunting Act of 2008 - Amends ...,"A bill to amend title 18, United States Code, ..."
3266,SECTION 1. SHORT TITLE.\n\n This Act may be...,Helping College Students Cross the Finish Line...,Helping College Students Cross the Finish Line...
3267,SECTION 1. SHORT TITLE.\n\n This Act may be...,Makes proceeds from such conveyances available...,Texas National Forests Improvement Act of 2000


In [9]:
import re

# first, get rid of line breaks
df["text"] = [re.sub("\n", "", df["text"][row]) for row in range(len(df))]
# Now moving onto parentheses and characters within
df["text"] = [re.sub(r'\([^)]*\)', '', df["text"][row]) for row in range(len(df))]
# Removing dashes
df["text"] = [re.sub("--", '', df["text"][row]) for row in range(len(df))]
# Removing extra spaces
df["text"] = [re.sub('\.(?!\s|\d|$)', '. ', df["text"][row]) for row in range(len(df))]
df["text"] = [re.sub("(\s+)", " ", df["text"][row]) for row in range(len(df))]

In [19]:
#!pip install transformers
#!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00


Google's T5 Model

In [20]:
# import pretrained model
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [21]:
# return_tensors: define return type (in this case it is pt, which is pytorch)
inputs = tokenizer.encode("summarize: " + df["text"][1], return_tensors = "pt", max_length = 4096, truncation = True)
inputs

tensor([[21603,    10,   466,  ...,   347,     5,     1]])

In [22]:
''' 
length_penalty: Exponential penalty to the length, where 1 is no penalty. so the higher the number, the longer the output
   
num_beams: allows the model to lookahead for four possible words, 
           to keep the most likely 4 of hypotheses at each time step, and choosing the one that has the overall highest probability.

early_stopping: set it to True so that generation is finished when all beam hypotheses reach the end of the string token.

# reference: https://www.thepythoncode.com/article/text-summarization-using-huggingface-transformers-python

'''

output = model.generate(inputs, min_length = 50, max_length = 300, length_penalty = 5.0, num_beams = 4, early_stopping = True)

In [23]:
print(tokenizer.decode(output[0]))

<pad> the federal forage fee may be cited as the 'federal forage Fee Act of 1993'' unless contrary to this statute, all grazing operations conducted on any Federal lands shall be subject to all applicable federal, state, and local laws. the federal forage fee shall apply to all authorized Federal AUMs under the jurisdiction of the United States Department of Agriculture and the United States Department of the Interior.</s>


Using Tensorflow

In [ ]:
'''
Attention mechanisms is a layer of neural networks added to deep learning models to focus their attention to specific parts of data, 
based on different weights assigned to different parts

reference: https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/#h-introduction-to-sequence-to-sequence-seq2seq-modeling
'''
from attention import Attention
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")